In [1]:
# +
from dataset.semi import SemiDataset
from model.semseg.deeplabv2 import DeepLabV2
from model.semseg.deeplabv3plus import DeepLabV3Plus
from model.semseg.pspnet import PSPNet
from utils import count_params, meanIOU, color_map, Accuracy, DiceCoefficient

from utilsf.loss_file import save_loss
# -

import argparse
from copy import deepcopy
import numpy as np
import os
from PIL import Image
import torch
from torch.nn import CrossEntropyLoss, DataParallel
from torch.optim import SGD
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
semi_setting = 'kidney/0114'
save_path = f'outdir/models/{semi_setting}'
pseudo_mask_path = f'outdir/pseudo_masks/{semi_setting}'
labeled_id_path = f'dataset/splits/{semi_setting}/labeled.txt'
reliable_id_path = f'outdir/reliable_ids/{semi_setting}'
data_root = '/home/S312112021/dataset/0_data_dataset_voc_950_kidney'
batch_size = 16
crop_size = 224
plus = True
dataset ='kidney'

if not os.path.exists(save_path):
    os.makedirs(save_path)
if not os.path.exists(pseudo_mask_path):
    os.makedirs(pseudo_mask_path)
if plus and reliable_id_path is None:
    exit('Please specify reliable-id-path in ST++.')

criterion = CrossEntropyLoss(ignore_index=255)

valset = SemiDataset(dataset, data_root, 'val', None)
valloader = DataLoader(valset, batch_size=4 if dataset == 'cityscapes' else 1,
                       shuffle=False, pin_memory=True, num_workers=4, drop_last=False)

In [3]:
MODE = 'train'

trainset = SemiDataset(dataset, data_root, MODE, crop_size, labeled_id_path)
trainset.ids = 2 * trainset.ids if len(trainset.ids) < 200 else trainset.ids
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True,
                         pin_memory=True, num_workers=16, drop_last=True)

In [4]:
for img, mask in trainloader:
    print('img shape: ', img.shape)
    print('mask shape: ', mask.shape)
    break

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/S312112021/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/S312112021/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/S312112021/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/S312112021/ST-PlusPlus/dataset/semi.py", line 109, in __getitem__
    img, mask = resize(img, mask, base_size, (0.5, 2.0))
  File "/home/S312112021/ST-PlusPlus/dataset/transform.py", line 50, in resize
    w, h = img.size
TypeError: cannot unpack non-iterable int object
